In [97]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0, 'src/models/')
sys.path.insert(0, 'src/')
sys.path.insert(0, '../conf')
sys.path.insert(0, '../')
sys.path.insert(0, 'conf/')
from utils import Dataset
from keras.callbacks import Callback
from keras.models import load_model
from keras import backend as K
pd.options.display.max_columns=999 
pd.options.display.max_rows=999

In [98]:
tr_queries = pd.read_csv('../../../input/kdd2019_regular/phase1/train_queries.csv')
tr_click = pd.read_csv('../../../input/kdd2019_regular/phase1/train_clicks.csv')
tr_plans = pd.read_csv('../../../input/kdd2019_regular/phase1/train_plans.csv')

In [99]:
len(list(set(tr_queries.sid) -set(tr_plans.sid)))  #--->8946个sid 直接是0  query 但是没有plans的用户

8946

In [100]:
tr_click[tr_click.sid.isin(list(set(tr_queries.sid) -set(tr_plans.sid)))]

,sid,click_time,click_mode


In [101]:
tr_data = tr_queries.merge(tr_click, on='sid', how='left')
tr_data = tr_data.merge(tr_plans, on='sid', how='left')
tr_data = tr_data.drop(['click_time'], axis=1)
tr_data['click_mode'] = tr_data['click_mode'].fillna(0)

In [103]:
from geopy.distance import geodesic
tr_data['num_o1'] = tr_data['o'].apply(lambda x: float(x.split(',')[0]))
tr_data['num_o2'] = tr_data['o'].apply(lambda x: float(x.split(',')[1]))
tr_data['num_d1'] = tr_data['d'].apply(lambda x: float(x.split(',')[0]))
tr_data['num_d2'] = tr_data['d'].apply(lambda x: float(x.split(',')[1]))
tr_data['num_direct_distance'] = tr_data.apply(lambda x: geodesic((x.num_o2, x.num_o1), (x.num_d2, x.num_d1)).m, axis=1)

In [384]:
def read_profile_data():
    profile_data = pd.read_csv('../../../input/kdd2019_regular/phase1/profiles.csv')
    profile_na = np.zeros(67)
    profile_na[0] = -1  # 增加pid=-1的
    profile_na = pd.DataFrame(profile_na.reshape(1, -1))
    profile_na.columns = profile_data.columns
    profile_data = profile_data.append(profile_na)
#     profile_data.columns = ['cat_' + i for i in profile_data.columns]
    return profile_data

In [385]:
profile = read_profile_data()

In [104]:
tr_data['num_direct_distance'].min()

0.0

In [105]:
tr_data[tr_data.pid.isnull()].shape,tr_data.shape

((163979, 13), (500000, 13))

In [106]:
check_zero_mode = tr_data[~tr_data.sid.isin(list(set(tr_queries.sid) -set(tr_plans.sid)))]

In [107]:
check_zero_mode.click_mode.value_counts()

2.0     136491
7.0      78209
1.0      70369
9.0      48864
5.0      47480
0.0      37718
3.0      24626
10.0     14882
4.0      12606
6.0      11863
11.0      6089
8.0       1857
Name: click_mode, dtype: int64

In [108]:
# 除去有querys 没有plans  可以看到0的mode还是很多  下面针对分析

In [109]:
check_zero_mode[(check_zero_mode.click_mode ==0)].pid.value_counts()

167770.0    302
112198.0    233
116844.0    146
203487.0    124
186452.0    113
129567.0     98
202427.0     93
109311.0     91
158443.0     87
129419.0     67
171246.0     66
170760.0     65
212590.0     65
199899.0     62
202643.0     59
147311.0     57
100011.0     52
163422.0     49
184718.0     46
199018.0     46
137947.0     45
202757.0     44
119374.0     43
191435.0     42
188871.0     41
197650.0     41
169231.0     39
127128.0     38
106441.0     36
162815.0     36
107147.0     36
115297.0     35
191445.0     35
131146.0     34
120773.0     33
160117.0     33
196999.0     33
104244.0     32
183581.0     32
196270.0     32
162056.0     32
126394.0     31
172775.0     31
111826.0     31
180178.0     31
170959.0     30
141409.0     29
167131.0     29
164539.0     28
207313.0     28
188372.0     28
112078.0     28
208890.0     28
213717.0     28
129866.0     26
101467.0     26
189676.0     26
117655.0     26
103276.0     26
135045.0     26
192310.0     24
173860.0     24
170341.0

In [110]:
tr_data[(tr_data.pid ==112198.0)].click_mode.value_counts()

2.0     1459
7.0      842
1.0      750
5.0      553
9.0      512
0.0      316
6.0      105
10.0      89
3.0       80
4.0       71
11.0      59
8.0       20
Name: click_mode, dtype: int64

In [111]:
check_zero_mode[(check_zero_mode.click_mode ==0)&(check_zero_mode.pid ==112198.0)].shape

(233, 13)

In [113]:
check_zero_mode[(check_zero_mode.click_mode ==0)&(check_zero_mode.pid ==112198.0)][:5].plans.values

array(['[{"distance": 761, "price": "", "eta": 720, "transport_mode": 5}, {"distance": 739, "price": "", "eta": 220, "transport_mode": 6}, {"distance": 1179, "price": "", "eta": 325, "transport_mode": 3}]',
       '[{"distance": 10119, "price": 400, "eta": 2570, "transport_mode": 2}, {"distance": 12756, "price": "", "eta": 1670, "transport_mode": 3}, {"distance": 12756, "price": 3500, "eta": 1970, "transport_mode": 4}, {"distance": 12144, "price": 300, "eta": 4423, "transport_mode": 1}]',
       '[{"distance": 5250, "price": 2000, "eta": 822, "transport_mode": 4}, {"distance": 5250, "price": "", "eta": 642, "transport_mode": 3}, {"distance": 5062, "price": "", "eta": 1531, "transport_mode": 6}, {"distance": 6186, "price": 400, "eta": 2150, "transport_mode": 1}, {"distance": 8410, "price": 500, "eta": 2910, "transport_mode": 7}]',
       '[{"distance": 2797, "price": 200, "eta": 1637, "transport_mode": 1}, {"distance": 2171, "price": "", "eta": 655, "transport_mode": 6}, {"distance": 51

In [118]:
check_zero_mode[(check_zero_mode.click_mode ==0)].describe()

,sid,pid,click_mode,num_o1,num_o2,num_d1,num_d2,num_direct_distance
count,3.771800e+04,22078.000000,37718.0,37718.000000,37718.000000,37718.000000,37718.000000,37718.000000
mean,1.713372e+06,158037.748573,0.0,116.390497,39.941517,116.388032,39.946118,12096.248020
std,1.004697e+06,33788.783970,0.0,0.130199,0.110974,0.139047,0.119777,13549.753517
min,2.000000e+00,100000.000000,0.0,115.500000,39.470000,115.440000,39.480000,0.000000
25%,8.161885e+05,128202.000000,0.0,116.310000,39.890000,116.320000,39.890000,2379.795186
50%,1.763793e+06,160807.000000,0.0,116.390000,39.930000,116.390000,39.930000,7957.994012
75%,2.592610e+06,186872.500000,0.0,116.450000,39.990000,116.450000,39.990000,16742.526470
max,3.396477e+06,216972.000000,0.0,117.270000,40.950000,117.370000,40.960000,177943.856426


In [116]:
check_zero_mode.describe()

,sid,pid,click_mode,num_o1,num_o2,num_d1,num_d2,num_direct_distance
count,4.910540e+05,330585.000000,491054.000000,491054.00000,491054.000000,491054.000000,491054.000000,491054.000000
mean,1.762291e+06,158226.628241,4.060908,116.38782,39.940439,116.384339,39.944215,12445.781222
std,1.015707e+06,33723.697840,3.090393,0.11601,0.101780,0.124606,0.109607,12432.745750
min,2.000000e+00,100000.000000,0.000000,115.47000,39.460000,115.440000,39.460000,0.000000
25%,8.106305e+05,127729.000000,2.000000,116.32000,39.890000,116.320000,39.890000,3439.071102
50%,1.848142e+06,161617.000000,3.000000,116.38000,39.930000,116.390000,39.920000,9045.657688
75%,2.663208e+06,187360.000000,7.000000,116.44000,39.990000,116.440000,39.990000,17231.230133
max,3.396588e+06,216977.000000,11.000000,117.29000,40.970000,117.370000,40.960000,177943.856426


In [119]:
# 距离上反应不出来跟mode 0的关系 --》整体和mode 0 没有太多的区分性
# 价格上应该也是类似
# 时间分布上粗略感觉没啥关系
# 区域上应该也没有
# user 上次点击分析，跟0好像也没啥关系
# user profile相关性分析？看哪个跟0有很强的相关性
# 跟plan分析，是否没点第一个就会点0？而不会点别的？---》这个feature已经有了，好像没啥用
# 

In [120]:
check_zero_mode = check_zero_mode.sort_values('req_time')

In [380]:
# 分析之前点过跟之后点过 0 mode的情况

In [122]:
fenxi_dt  = '2018-11-01'

In [341]:
before_zero_mode = check_zero_mode[check_zero_mode.req_time <=fenxi_dt]

after_zero_mode = check_zero_mode[check_zero_mode.req_time >fenxi_dt]

In [342]:
before_zero_mode_contains = before_zero_mode.groupby('pid').click_mode.apply(lambda x: 1 if 0.0 in x.tolist() else -1).reset_index()

In [343]:
before_zero_mode_contains.head()

,pid,click_mode
0,100000.0,1
1,100002.0,-1
2,100005.0,1
3,100006.0,1
4,100007.0,-1


In [344]:
after_zero_mode[after_zero_mode.pid.isin(before_zero_mode_contains.pid)].click_mode.value_counts()

2.0     36840
7.0     21729
1.0     18739
5.0     11484
9.0      8465
0.0      7137
3.0      4122
10.0     3425
4.0      2478
6.0      2290
11.0     1135
8.0       633
Name: click_mode, dtype: int64

In [346]:
#之前点击过0的，跟之后点击过0的目前来看没啥关系
after_zero_mode[after_zero_mode.pid.isin(before_zero_mode_contains[before_zero_mode_contains.click_mode==1].pid)].click_mode.value_counts()

2.0     25422
7.0     15172
1.0     13063
5.0      8381
9.0      5969
0.0      4742
3.0      2685
10.0     2265
4.0      1610
6.0      1590
11.0      813
8.0       437
Name: click_mode, dtype: int64

In [347]:
after_zero_mode[after_zero_mode.pid.isin(before_zero_mode_contains[before_zero_mode_contains.click_mode==1].pid)].head()

,sid,pid,req_time,o,d,click_mode,plan_time,plans,num_o1,num_o2,num_d1,num_d2,num_direct_distance
106901,2594832,177969.0,2018-11-01 00:07:34,"116.37,39.98","116.48,40.96",0.0,2018-11-01 00:07:34,"[{""distance"": 153516, ""price"": 51000, ""eta"": 9...",116.37,39.98,116.48,40.96,109221.883540
52539,2617332,120497.0,2018-11-01 00:09:01,"116.40,40.07","116.40,40.08",5.0,2018-11-01 00:09:01,"[{""distance"": 782, ""price"": """", ""eta"": 668, ""t...",116.40,40.07,116.40,40.08,1110.360742
168610,2585773,176540.0,2018-11-01 00:09:52,"116.24,39.90","116.44,39.96",1.0,2018-11-01 00:09:52,"[{""distance"": 27309, ""price"": 600, ""eta"": 7087...",116.24,39.90,116.44,39.96,18348.351087
83945,2601818,112198.0,2018-11-01 00:10:24,"116.34,39.99","116.38,39.98",1.0,2018-11-01 00:10:24,"[{""distance"": 5391, ""price"": 200, ""eta"": 3151,...",116.34,39.99,116.38,39.98,3592.401036
134042,2620190,213660.0,2018-11-01 00:12:09,"116.40,39.87","116.32,39.89",1.0,2018-11-01 00:12:09,"[{""distance"": 14358, ""price"": 300, ""eta"": 4705...",116.40,39.87,116.32,39.89,7194.727302


In [348]:
# 下面来看还是跟距离没啥关系

In [349]:
after_zero_mode[(after_zero_mode.click_mode == 0)&after_zero_mode.pid.isin(before_zero_mode_contains[before_zero_mode_contains.click_mode==1].pid)].num_direct_distance.describe()

count      4742.000000
mean      11990.851132
std       13241.165223
min           0.000000
25%        2379.425378
50%        7958.114959
75%       16970.193468
max      169535.589913
Name: num_direct_distance, dtype: float64

In [350]:
after_zero_mode[(after_zero_mode.click_mode == 0)&after_zero_mode.pid.isin(before_zero_mode_contains[before_zero_mode_contains.click_mode==-1].pid)].num_direct_distance.describe()

count      2395.000000
mean      11708.063102
std       12863.879899
min           0.000000
25%        2379.178686
50%        7819.194972
75%       16389.105162
max      132438.755404
Name: num_direct_distance, dtype: float64

# user是否经常使用app分析

In [353]:
large_mode = lambda x:'gongjiao_ditie' if x in [1,2,5,7,9,10,11] else('zijia' if x in [3]  \
                                                                else('dache' if x in [4,8] \
                                                                else 'meidianji'))

In [354]:
after_zero_mode['large_mode'] = after_zero_mode.click_mode.apply(large_mode)

In [357]:
after_zero_mode.pid.unique()

array([    nan, 177969., 172242., ..., 190374., 203167., 157196.])

In [359]:
after_zero_mode.pid = after_zero_mode.pid.fillna(-1)

In [360]:
after_zero_mode[after_zero_mode.pid ==103073.0]

,sid,pid,req_time,o,d,click_mode,plan_time,plans,num_o1,num_o2,num_d1,num_d2,num_direct_distance,large_mode
483653,2574146,103073.0,2018-11-01 11:36:17,"116.29,39.83","116.29,39.82",2.0,2018-11-01 11:36:17,"[{""distance"": 976, ""price"": 300, ""eta"": 414, ""...",116.29,39.83,116.29,39.82,1110.312715,gongjiao_ditie
170579,2574148,103073.0,2018-11-01 13:06:26,"116.29,39.82","116.37,39.88",2.0,2018-11-01 13:06:26,"[{""distance"": 15134, ""price"": 500, ""eta"": 2452...",116.29,39.82,116.37,39.88,9552.727823,gongjiao_ditie
435996,2574145,103073.0,2018-11-01 13:08:13,"116.29,39.82","116.37,39.88",2.0,2018-11-01 13:08:13,"[{""distance"": 15182, ""price"": 500, ""eta"": 2493...",116.29,39.82,116.37,39.88,9552.727823,gongjiao_ditie
436774,509465,103073.0,2018-11-09 00:23:56,"116.34,40.09","116.39,39.91",4.0,2018-11-09 00:23:56,"[{""distance"": 27664, ""price"": 600, ""eta"": 8629...",116.34,40.09,116.39,39.91,20437.215443,dache
364820,3182096,103073.0,2018-11-21 05:37:28,"116.46,39.92","116.61,40.05",3.0,2018-11-21 05:37:28,"[{""distance"": 23310, ""price"": 2800, ""eta"": 282...",116.46,39.92,116.61,40.05,19300.202241,zijia
432181,1200308,103073.0,2018-11-22 10:02:23,"116.46,39.92","116.46,39.93",5.0,2018-11-22 10:02:23,"[{""distance"": 1102, ""price"": """", ""eta"": 942, ""...",116.46,39.92,116.46,39.93,1110.331916,gongjiao_ditie
368526,523581,103073.0,2018-11-23 18:35:08,"116.36,39.87","116.21,39.97",7.0,2018-11-23 18:35:08,"[{""distance"": 25375, ""price"": 700, ""eta"": 6169...",116.36,39.87,116.21,39.97,16962.860942,gongjiao_ditie
369567,844106,103073.0,2018-11-26 10:08:31,"116.31,39.84","116.21,39.97",7.0,2018-11-26 10:08:31,"[{""distance"": 23676, ""price"": 700, ""eta"": 4870...",116.31,39.84,116.21,39.97,16777.099128,gongjiao_ditie
463078,844102,103073.0,2018-11-26 11:45:37,"116.37,39.86","116.21,39.97",7.0,2018-11-26 11:45:37,"[{""distance"": 26900, ""price"": 700, ""eta"": 5877...",116.37,39.86,116.21,39.97,18338.848719,gongjiao_ditie
295039,844105,103073.0,2018-11-26 12:26:51,"116.33,39.94","116.21,39.97",1.0,2018-11-26 12:26:51,"[{""distance"": 14709, ""price"": 300, ""eta"": 3662...",116.33,39.94,116.21,39.97,10781.459049,gongjiao_ditie


In [361]:
see = pd.DataFrame(after_zero_mode.groupby('pid').large_mode.value_counts())

In [ ]:
after_zero_mode.grou

In [362]:
see.columns = ['mode_counts']

In [363]:
see = see.reset_index()

In [364]:
see[see.pid ==103073.0]

,pid,large_mode,mode_counts
1016,103073.0,gongjiao_ditie,12
1017,103073.0,zijia,2
1018,103073.0,dache,1


In [365]:
see_all = see.groupby('pid').mode_counts.sum().reset_index()

In [366]:
merge = pd.merge(see,see_all,on='pid')

In [367]:
merge.large_mode.unique()

array(['gongjiao_ditie', 'meidianji', 'zijia', 'dache'], dtype=object)

In [368]:
merge[(merge.mode_counts_y > 5)].shape # 选过5次以上的用户

(10588, 4)

In [369]:
merge[(merge.mode_counts_y > 5)&(merge.mode_counts_x > 1)].shape,merge[(merge.mode_counts_y > 5)&(merge.mode_counts_x <= 1)].shape

((7283, 4), (3305, 4))

In [370]:
merge[(merge.mode_counts_y > 4)&(merge.mode_counts_x > 1)].shape,merge[(merge.mode_counts_y > 4)&(merge.mode_counts_x <= 1)].shape

((8507, 4), (3867, 4))

In [371]:
merge[(merge.mode_counts_y > 3)&(merge.mode_counts_x > 1)].shape,merge[(merge.mode_counts_y > 3)&(merge.mode_counts_x <= 1)].shape

((10189, 4), (4602, 4))

In [ ]:
# -->可以看出随着用户点击次数越多，用户约倾向于选择跟之前一样的；

In [375]:
merge[merge.large_mode=='meidianji']

,pid,large_mode,mode_counts_x,mode_counts_y
1,-1.0,meidianji,8166,72825
6,100005.0,meidianji,1,15
11,100011.0,meidianji,41,422
16,100033.0,meidianji,2,3
22,100045.0,meidianji,1,3
23,100047.0,meidianji,1,1
26,100050.0,meidianji,1,10
28,100051.0,meidianji,2,3
31,100052.0,meidianji,1,8
36,100063.0,meidianji,1,6


In [376]:
merge[(merge.large_mode =='zijia')]

,pid,large_mode,mode_counts_x,mode_counts_y
2,-1.0,zijia,4785,72825
12,100011.0,zijia,20,422
27,100050.0,zijia,1,10
37,100063.0,zijia,1,6
41,100072.0,zijia,4,19
45,100079.0,zijia,1,2
48,100080.0,zijia,1,17
64,100111.0,zijia,1,8
81,100167.0,zijia,1,4
87,100191.0,zijia,1,1


In [204]:
check = pd.pivot_table(see,columns=['click_mode'],values=['mode_counts'],index='pid').reset_index()

In [220]:
check.shape

(27044, 13)

In [222]:
check.columns = ['pid']+['cat_click_mode_'+str(i) for i in range(12)]

In [223]:
check.head()

,pid,cat_click_mode_0,cat_click_mode_1,cat_click_mode_2,cat_click_mode_3,cat_click_mode_4,cat_click_mode_5,cat_click_mode_6,cat_click_mode_7,cat_click_mode_8,cat_click_mode_9,cat_click_mode_10,cat_click_mode_11
0,100000.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005.0,1.0,NaN,7.0,NaN,NaN,1.0,NaN,3.0,NaN,2.0,NaN,1.0
2,100007.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,100010.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
4,100011.0,32.0,67.0,134.0,20.0,11.0,36.0,9.0,59.0,1.0,32.0,16.0,5.0


In [381]:
# 分析user profile 

In [386]:
profile.head()

,pid,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,p26,p27,p28,p29,p30,p31,p32,p33,p34,p35,p36,p37,p38,p39,p40,p41,p42,p43,p44,p45,p46,p47,p48,p49,p50,p51,p52,p53,p54,p55,p56,p57,p58,p59,p60,p61,p62,p63,p64,p65
0,196356.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,204083.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,170667.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,115511.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,129719.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [388]:
merge_profile = pd.merge(check_zero_mode,profile,on='pid')

In [389]:
merge_profile.head()

,sid,pid,req_time,o,d,click_mode,plan_time,plans,num_o1,num_o2,num_d1,num_d2,num_direct_distance,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p24,p25,p26,p27,p28,p29,p30,p31,p32,p33,p34,p35,p36,p37,p38,p39,p40,p41,p42,p43,p44,p45,p46,p47,p48,p49,p50,p51,p52,p53,p54,p55,p56,p57,p58,p59,p60,p61,p62,p63,p64,p65
0,753082,101467.0,2018-10-01 00:00:10,"116.34,39.90","116.34,39.90",5.0,2018-10-01 00:00:10,"[{""distance"": 923, ""price"": """", ""eta"": 859, ""t...",116.34,39.90,116.34,39.90,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,722886,101467.0,2018-10-01 01:46:06,"116.39,39.87","116.39,39.91",1.0,2018-10-01 01:46:06,"[{""distance"": 6134, ""price"": 200, ""eta"": 2932,...",116.39,39.87,116.39,39.91,4441.300778,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,785617,101467.0,2018-10-01 02:25:19,"116.37,39.86","116.39,39.91",1.0,2018-10-01 02:25:19,"[{""distance"": 7200, ""price"": 200, ""eta"": 3129,...",116.37,39.86,116.39,39.91,5809.227490,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,785618,101467.0,2018-10-01 02:37:46,"116.37,39.89","116.39,39.91",3.0,2018-10-01 02:37:46,"[{""distance"": 4759, ""price"": 200, ""eta"": 2743,...",116.37,39.89,116.39,39.91,2802.972797,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,753079,101467.0,2018-10-01 07:58:52,"116.34,39.90","116.39,39.78",1.0,2018-10-01 07:58:52,"[{""distance"": 17332, ""price"": 2200, ""eta"": 513...",116.34,39.90,116.39,39.78,13994.235019,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [407]:
value.columns = ['']

Index([u'click_mode'], dtype='object')

In [430]:
df_all=[]
for i in range(profile.shape[1]-1):
    print 'p'+str(i)
    value = pd.DataFrame(merge_profile.groupby('p'+str(i)).click_mode.value_counts())
    value.columns = ['mode_counts']
    value = value.reset_index()
    value.rename(columns={'p'+str(i):'p_value'},inplace=True)
    value['profile']='p'+str(i)
    print value
    df_all.append(value)
    print '--------------------'
df_all = pd.concat(df_all)

p0
    p_value  click_mode  mode_counts profile
0       0.0         2.0        43752      p0
1       0.0         7.0        24492      p0
2       0.0         1.0        21761      p0
3       0.0         9.0        14456      p0
4       0.0         0.0        13126      p0
5       0.0         5.0        12109      p0
6       0.0         3.0         9511      p0
7       0.0        10.0         5059      p0
8       0.0         4.0         4799      p0
9       0.0         6.0         3628      p0
10      0.0        11.0         1746      p0
11      0.0         8.0          646      p0
12      1.0         2.0        52564      p0
13      1.0         7.0        31592      p0
14      1.0         1.0        27736      p0
15      1.0         9.0        18542      p0
16      1.0         5.0        17942      p0
17      1.0         0.0         8952      p0
18      1.0        10.0         4123      p0
19      1.0         3.0         4072      p0
20      1.0         6.0         3999      p0
21     

    p_value  click_mode  mode_counts profile
0       0.0         2.0        54407     p10
1       0.0         7.0        31109     p10
2       0.0         1.0        26816     p10
3       0.0         9.0        17946     p10
4       0.0         5.0        15003     p10
5       0.0         0.0        14433     p10
6       0.0         3.0         9907     p10
7       0.0        10.0         5674     p10
8       0.0         4.0         5292     p10
9       0.0         6.0         4251     p10
10      0.0        11.0         2198     p10
11      0.0         8.0          711     p10
12      1.0         2.0        41909     p10
13      1.0         7.0        24975     p10
14      1.0         1.0        22681     p10
15      1.0         9.0        15052     p10
16      1.0         5.0        15048     p10
17      1.0         0.0         7645     p10
18      1.0         3.0         3676     p10
19      1.0        10.0         3508     p10
20      1.0         6.0         3376     p10
21      1.

    p_value  click_mode  mode_counts profile
0       0.0         2.0        95048     p18
1       0.0         7.0        55425     p18
2       0.0         1.0        48833     p18
3       0.0         9.0        32415     p18
4       0.0         5.0        29686     p18
5       0.0         0.0        21684     p18
6       0.0         3.0        13275     p18
7       0.0        10.0         9060     p18
8       0.0         4.0         7718     p18
9       0.0         6.0         7491     p18
10      0.0        11.0         4075     p18
11      0.0         8.0         1167     p18
12      1.0         2.0         1268     p18
13      1.0         1.0          664     p18
14      1.0         7.0          659     p18
15      1.0         9.0          583     p18
16      1.0         0.0          394     p18
17      1.0         5.0          365     p18
18      1.0         3.0          308     p18
19      1.0         4.0          142     p18
20      1.0         6.0          136     p18
21      1.

    p_value  click_mode  mode_counts profile
0       0.0         2.0        69239     p26
1       0.0         7.0        41763     p26
2       0.0         1.0        36981     p26
3       0.0         9.0        24752     p26
4       0.0         5.0        22293     p26
5       0.0         0.0        17435     p26
6       0.0         3.0        11171     p26
7       0.0        10.0         7044     p26
8       0.0         4.0         6089     p26
9       0.0         6.0         5847     p26
10      0.0        11.0         3246     p26
11      0.0         8.0          951     p26
12      1.0         2.0        27077     p26
13      1.0         7.0        14321     p26
14      1.0         1.0        12516     p26
15      1.0         9.0         8246     p26
16      1.0         5.0         7758     p26
17      1.0         0.0         4643     p26
18      1.0         3.0         2412     p26
19      1.0        10.0         2138     p26
20      1.0         6.0         1780     p26
21      1.

    p_value  click_mode  mode_counts profile
0       0.0         2.0        69397     p36
1       0.0         7.0        41429     p36
2       0.0         1.0        35900     p36
3       0.0         9.0        22962     p36
4       0.0         5.0        21170     p36
5       0.0         0.0        15255     p36
6       0.0         3.0         8868     p36
7       0.0        10.0         6233     p36
8       0.0         4.0         5461     p36
9       0.0         6.0         5293     p36
10      0.0        11.0         3028     p36
11      0.0         8.0          796     p36
12      1.0         2.0        26919     p36
13      1.0         7.0        14655     p36
14      1.0         1.0        13597     p36
15      1.0         9.0        10036     p36
16      1.0         5.0         8881     p36
17      1.0         0.0         6823     p36
18      1.0         3.0         4715     p36
19      1.0        10.0         2949     p36
20      1.0         4.0         2399     p36
21      1.

    p_value  click_mode  mode_counts profile
0       0.0         2.0        81655     p47
1       0.0         7.0        47370     p47
2       0.0         1.0        41454     p47
3       0.0         9.0        27621     p47
4       0.0         5.0        25422     p47
5       0.0         0.0        18939     p47
6       0.0         3.0        11700     p47
7       0.0        10.0         7870     p47
8       0.0         4.0         6780     p47
9       0.0         6.0         6478     p47
10      0.0        11.0         3476     p47
11      0.0         8.0          990     p47
12      1.0         2.0        14661     p47
13      1.0         7.0         8714     p47
14      1.0         1.0         8043     p47
15      1.0         9.0         5377     p47
16      1.0         5.0         4629     p47
17      1.0         0.0         3139     p47
18      1.0         3.0         1883     p47
19      1.0        10.0         1312     p47
20      1.0         6.0         1149     p47
21      1.

    p_value  click_mode  mode_counts profile
0       0.0         2.0        92150     p56
1       0.0         7.0        53723     p56
2       0.0         1.0        47496     p56
3       0.0         9.0        31685     p56
4       0.0         5.0        28927     p56
5       0.0         0.0        21107     p56
6       0.0         3.0        12992     p56
7       0.0        10.0         8791     p56
8       0.0         4.0         7487     p56
9       0.0         6.0         7377     p56
10      0.0        11.0         3983     p56
11      0.0         8.0         1117     p56
12      1.0         2.0         4166     p56
13      1.0         7.0         2361     p56
14      1.0         1.0         2001     p56
15      1.0         9.0         1313     p56
16      1.0         5.0         1124     p56
17      1.0         0.0          971     p56
18      1.0         3.0          591     p56
19      1.0        10.0          391     p56
20      1.0         4.0          373     p56
21      1.

    p_value  click_mode  mode_counts profile
0       0.0         2.0        95810     p64
1       0.0         7.0        55794     p64
2       0.0         1.0        49238     p64
3       0.0         9.0        32831     p64
4       0.0         5.0        29925     p64
5       0.0         0.0        21922     p64
6       0.0         3.0        13471     p64
7       0.0        10.0         9120     p64
8       0.0         4.0         7805     p64
9       0.0         6.0         7563     p64
10      0.0        11.0         4119     p64
11      0.0         8.0         1162     p64
12      1.0         2.0          506     p64
13      1.0         7.0          290     p64
14      1.0         1.0          259     p64
15      1.0         9.0          167     p64
16      1.0         0.0          156     p64
17      1.0         5.0          126     p64
18      1.0         3.0          112     p64
19      1.0         6.0           64     p64
20      1.0        10.0           62     p64
21      1.

In [436]:
for i in range(profile.shape[1]-1):
    print 'p'+str(i)
    print 'ratio is ',float(df_all[(df_all.profile=='p'+str(i))&(df_all.p_value ==0)].mode_counts.sum()) / df_all[(df_all.profile=='p'+str(i))&(df_all.p_value ==1)].mode_counts.sum()
    print 'ratio is ',float(df_all[(df_all.profile=='p'+str(i))&(df_all.click_mode==0)&(df_all.p_value ==0)].mode_counts.values[0]) / df_all[(df_all.profile=='p'+str(i))&(df_all.click_mode==0)&(df_all.p_value ==1)].mode_counts.values[0]

p0
ratio is  0.883675213675
ratio is  1.46626452189
p1
ratio is  40.3748435544
ratio is  39.8096118299
p2
ratio is  2.87088275588
ratio is  2.29620782323
p3
ratio is  6.55312100165
ratio is  4.58230088496
p4
ratio is  19.4874194348
ratio is  12.2441511698
p5
ratio is  215.77704918
ratio is  95.8333333333
p6
ratio is  1703.04639175
ratio is  711.193548387
p7
ratio is  1.67827629788
ratio is  1.78375992939
p8
ratio is  0.596381178652
ratio is  0.561275723075
p9
ratio is  2.6508155625
ratio is  1.90614716335
p10
ratio is  1.31440513029
ratio is  1.88790058862
p11
ratio is  122.768251591
ratio is  167.534351145
p12
ratio is  1668.62121212
ratio is  2758.75
p13
ratio is  9.47348244836
ratio is  13.0356007629
p14
ratio is  432.838582677
ratio is  550.95
p15
ratio is  144.439947206
ratio is  118.989130435
p16
ratio is  81.1329192547
ratio is  75.3944636678
p17
ratio is  40.3541406055
ratio is  33.87835703
p18
ratio is  69.2177145285
ratio is  55.0355329949
p19
ratio is  153.334733894
ratio is

In [435]:
# p13 29 33 9  6  5 0 ---> 这几个类别0分的还挺开的；

 ratio is  329210.0


In [491]:
tr_click.head()

,sid,click_time,click_mode
0,2848914,2018-11-17 18:42:17,1
1,2629085,2018-10-12 16:28:13,3
2,602598,2018-11-11 16:38:42,2
3,2022975,2018-10-14 12:28:11,9
4,988425,2018-11-12 16:47:30,7


In [437]:
# 构建user feature

In [462]:
check_zero_mode['req_time'] = pd.to_datetime(check_zero_mode['req_time'])

In [492]:
df_test=[]
for index,group in check_zero_mode.groupby('pid'):
    print index
    group['cat_last_click_mode'] = group.click_mode.shift(1)
    group['cat_last_click_mode'] = group['cat_last_click_mode'].fillna(0)
    group['num_last_req_time'] = group.req_time.shift(1)
    group['num_last_req_time'].fillna(1000)
    group['num_how_long_till_this_time'] = group.req_time.astype(int) - group['num_last_req_time'].astype(int)
    group['num_hours_till_this_time'] = group.req_time - group['num_last_req_time']

    
    group = group.reset_index(drop=True)
    till_now_mode_clicks=np.zeros((group.shape[0], 12))
    for i,row in group.iterrows():
        print i
        last_time = group[group.req_time <= row['req_time']].cat_last_click_mode.value_counts()
        for j,v in zip(last_time.index,last_time.values):
            till_now_mode_clicks[i,int(j)]=v

    for i in range(11):
        group['p'+str(i)+'_till_now_mode_clicks'] = till_now_mode_clicks[:,i]

    df_test.append(group)
pd.concat(df_test)

100000.0
0
1
2
3
4
5
6
7
100002.0
0
100005.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
100006.0
0
1
2
3
4
100007.0
0
1
2
100010.0
0
1
100011.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
2

KeyboardInterrupt: 

In [494]:
tr_queries.pid.unique().shape

(46192,)

In [486]:
till_now_mode_clicks[:]

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [460]:
for i,v in zip(last_time.index,last_time.values):
    print i,v
    till_now_mode_clicks[i,]

9.0 1
7.0 1


[(9.0, 1), (7.0, 1)]

# 意外发现

In [67]:
# 规则如果是一个就是5或者0
# 
check_zero_mode[(check_zero_mode.o ==check_zero_mode.d)].click_mode.value_counts()

5.0     21662
0.0      2552
6.0      1216
3.0       831
1.0       318
2.0       112
4.0        53
7.0        18
9.0         6
10.0        2
8.0         1
Name: click_mode, dtype: int64

In [89]:
check_zero_mode[(check_zero_mode.o ==check_zero_mode.d)].head()

,sid,pid,req_time,o,d,click_mode,plan_time,plans,num_o1,num_o2,num_d1,num_d2,num_direct_distance
69,1522904,170410.0,2018-11-28 15:39:33,"116.31,40.07","116.31,40.07",5.0,2018-11-28 15:39:33,"[{""distance"": 15, ""price"": """", ""eta"": 12, ""tra...",116.31,40.07,116.31,40.07,6.284908e+06
75,1847932,108719.0,2018-11-03 11:16:24,"116.34,39.95","116.34,39.95",5.0,2018-11-03 11:16:24,"[{""distance"": 148, ""price"": """", ""eta"": 126, ""t...",116.34,39.95,116.34,39.95,6.308357e+06
79,2143984,NaN,2018-10-19 20:11:43,"116.45,39.93","116.45,39.93",5.0,2018-10-19 20:11:43,"[{""distance"": 73, ""price"": """", ""eta"": 62, ""tra...",116.45,39.93,116.45,39.93,6.320277e+06
96,1975681,NaN,2018-10-11 16:37:17,"116.43,39.92","116.43,39.92",5.0,2018-10-11 16:37:17,"[{""distance"": 47, ""price"": """", ""eta"": 40, ""tra...",116.43,39.92,116.43,39.92,6.320517e+06
138,201739,NaN,2018-10-20 06:49:32,"116.49,39.91","116.49,39.91",0.0,2018-10-20 06:49:32,"[{""distance"": 452, ""price"": """", ""eta"": 386, ""t...",116.49,39.91,116.49,39.91,6.326859e+06


In [253]:
geodesic((40.07,116.31), (40.04,116.31)).m

3331.070691544427

In [ ]:
#2 用户选择的次数越多，之前